### 케라스의 레이어(Layer) 객체

#### 1. 케라스 레이어

#### 2. 드롭아웃(Dropout) 레이어
- 입력값 중에서 일정 비율(rate)만큼 임의로 선택하고, 선택한 입력값을 강제로 0으로 설정함으로써 유닛의 무효화 처리 구현

In [1]:
# 드롭아웃 레이어 사용 예

# 드롭아웃 레이어를 포함하는 Sequential 모델 예제
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(64, activation = 'relu', input_dim = 20))
# Dropout을 rate = 0.5로 적용
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'SGD',
    metrics = ['accuracy']
)

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### 3. BatchNormalization(배치정규화) 레이어
- 배치마다 다음 레이어로의 입력값을 정규화

In [2]:
# BatchNormalization 레이어 사용 예

# BatchNormalization 층을 포함하는 Sequential 모델 예제
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, BatchNormalization

model = Sequential()
model.add(Dense(64, input_dim=20))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'SGD',
    metrics = ['accuracy']
)

#### 4. Lambda(람다) 레이어
- 임의의 식과 함수를 케라스 레이어 객체로 네트워크에 추가

In [3]:
# Lambda 레이어 사용 예

# Lambda 레이어 사용 예제
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Lambda

model_in = Input(shape = (20, ))
# 모든 입력값을 255로 나누는 변환
x = Lambda(lambda x : x/255.)(model_in)
x = Dense(64, activation = 'relu')(x)
model_out = Dense(10, activation = 'softmax')(x)

model = Model(inputs = model_in, outputs = model_out)
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'SGD',
    metrics = ['accuracy']
)

### 활성화 함수(Activation)

- 케라스에서 활성화 함수를 추가하는 방법
    - 레이어 객체에서 activation의 인수로 지정하는 방법
    - 별도로 개별 Activation 레이어를 호출해서 명시적으로 네트워크에 추가하는 방법

In [4]:
# 1) 레이어의 인수를 사용해서 활성화 함수를 추가하는 예

# Dense 레이어의 인수로 relu와 시그모이드를 지정해서 활성화 함수를 추가
model = Sequential()
model.add(Dense(64, activation = 'relu', input_dim = 20))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'SGD',
    metrics = ['accuracy']
)

In [5]:
# 2) Activation 레이어를 호출해서 활성화 함수를 생성/추가하는 예

# Activation 레이어를 호출해서 별도로 활성화 층을 추가
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.activations import relu

model = Sequential()
model.add(Dense(64, input_dim=20))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'SGD',
    metrics = ['accuracy']
)

#### ReLU(Rectified Linear Unit)
- 입력값이 0 이하인 경우에는 0, 0보다 큰 경우에는 입력값을 그대로 출력
- 시그모이드 함수는 입력값이 커질수록 경사가 작아졌지만, ReLU는 일정하기 때문에 경사 소실 문제를 완화할 수 있다

#### LeakyReLU
- 유닛이 활성화되지 않을 때(x<=0)에도 아주 작은 경사를 가진다
- 이것으로 경사 소실을 방지하고, 학습 속도를 향상시키는 효과를 기대할 수 있다
- DCGAN(Deep Convolutional Generative Adversarial Network) 모델에 자주 사용

#### ELU(Exponential Linear Unit)
- 마이너스의 입력에 대해서 지수 함수에서 1을 뺀 값을 적용

#### Clipped ReLU
- ReLU의 출력값이 일정 크기 이상이 되지 않도록 변형한 것
- 입력에 대해서 출력이 한계가 있는 상태(Clipping) 

In [6]:
# Clipped ReLU의 예
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.activations import relu

model = Sequential()
model.add(
    Dense(
        64,
        input_dim = 20,
        activation = lambda x : relu(x, max_value = 2)
    )
)
model.add(Dense(1, activation = 'sigmoid'))

model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'SGD',
    metrics = ['accuracy']
)

### ImageDataGenerator
- 입력할 이미지를 효율적으로 처리하기 위한 생성기
- 전처리를 실시간으로 실시
- 미니 배치 크기 단위로 모델에 데이터를 쉽게 전달

In [9]:
# 생성기 생성
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator 생성
# 대표적인 옵션을 지정한 예
gen = ImageDataGenerator(
    # 스케일 변환
    rescale = 1/255.,
    # 데이터 확장 관련
    rotation_range = 90.,
    width_shift_range = 1.,
    height_shift_range = .5,
    shear_range = .8,
    zoom_range = .5,
    horizontal_flip = True,
    vertical_flip = True
)

In [11]:
# ImageDataGenerator를 생성한 후에는
# flow 메서드와 flow_from_directory 메서드를 호출해서
# 지정한 처리가 적용된 이미지를 미니 배치 단위로 출력하는 iterator 취득

# iterator의 생성과 데이터 취득

# 폴더에서 이미지를 읽어들이고, iterator 생성
iters = gen.flow_from_directory(
    'img_7',
    target_size = (32, 32),
    class_mode = 'binary',
    batch_size = 5,
    shuffle = True
)

# iterator로부터 1 미니 배치분의 데이터를 취득
x_train_batch, y_train_batch = next(iters)

print('shape of x_train_batch : ', x_train_batch.shape)
print('shape of y_train_batch : ', y_train_batch.shape)

Found 25 images belonging to 2 classes.
shape of x_train_batch :  (5, 32, 32, 3)
shape of y_train_batch :  (5,)


In [12]:
# iterator를 이용한 모델 학습
import math
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Conv2D

# 분류용 모델 구축
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(32, 32, 3)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'rmsprop'
)

# 몇 미니 배치를 학습하면 1에폭인지 계산
steps_per_epoch = math.ceil(iters.samples/5)

# fit_generator 메서드에 iterator 전달
model.fit_generator(
    iters,
    steps_per_epoch = steps_per_epoch
)

Instructions for updating:
Use tf.cast instead.
5/5 [==============================] - 7s 1s/step - loss: 2.7339
